### Notebook for training the hand gesture classification

In [1]:
#Import dependencies

import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots


RANDOM_SEED = 42

In [2]:
#Path selection and definition of constant

dataset = '../assets/dataset/keypoint.csv'
model_save_path = '../assets/models'
tflite_save_path = '../assets/models/tf_LiteModel.tflite'

epoche = 1000
batch_size = 128
num_classes = 6

### Dataset Reading

In [ ]:
# Alternative way to split dataset in test_set and train_set

x_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, train_size=0.70, random_state=RANDOM_SEED)

In [9]:
# split dataset in 60% train_set, 20% test_set and 20% validation_set

df = pd.read_csv(dataset)

train_dataset, test_dataset, validation_dataset = np.split(df.sample(frac=1, random_state=RANDOM_SEED), 
                       [int(.6*len(df)), int(.8*len(df))])

# x_train y_train 
x_train = np.array(train_dataset.iloc[:, 1:])
y_train = np.array(train_dataset.iloc[:, 0])

# x_test y_test
x_test = np.array(test_dataset.iloc[:, 1:])
y_test = np.array(test_dataset.iloc[:, 0])

# x_validation y_validation
x_validation = np.array(validation_dataset.iloc[:, 1:])
y_validation = np.array(validation_dataset.iloc[:, 0])                       


### Data Plot


In [10]:
def dict_value(y_dataset):
    dict = {}
    for element in y_dataset:
        if element in dict:
            valore = dict[element]
            valore = valore + 1
            dict[element] = valore
        else :
            dict[element] = 1    
    return dict

In [11]:
dict_train = dict_value(y_train)
dict_test = dict_value(y_test)
dict_validation = dict_value(y_validation) 


fig = go.Figure(data=[
    go.Bar(name = 'Palm open',x=['train data', ' test data', 'convalidation data'], y=[dict_train[0], dict_test[0], dict_validation[0]]),
    go.Bar(name = 'Fist',x= ['train data', ' test data', 'convalidation data'], y=[dict_train[1], dict_test[1], dict_validation[1]]),
    go.Bar(name = 'Index',x= ['train data', ' test data', 'convalidation data'], y=[dict_train[2], dict_test[2], dict_validation[2]]),
    go.Bar(name = 'Finger 2',x= ['train data', ' test data', 'convalidation data'], y=[dict_train[3], dict_test[3], dict_validation[3]]),
    go.Bar(name = 'Finger 3',x= ['train data', ' test data', 'convalidation data'], y=[dict_train[4], dict_test[4], dict_validation[4]]),
    go.Bar(name = 'Finger 4',x= ['train data', ' test data', 'convalidation data'], y=[dict_train[5], dict_test[5], dict_validation[5]]),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_yaxes(type = "log")
pyo.iplot(fig,filename = 'split data')
fig.write_html("../template/img/data-plot.html")

### Model Building

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 6)                 66        
                                                                 
Total params: 1,136
Trainable params: 1,136
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
model_save_path+'/model.h5',verbose=1, save_weights_only=False, save_best_only= True, mode='auto')
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

# Model compilation
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 6)                 66        
                                                                 
Total params: 1,136
Trainable params: 1,136
Non-trainable params: 0
_________________________________________________________________


### Training

In [14]:
history = model.fit(
    x_train,
    y_train,
    epochs=epoche,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[cp_callback, es_callback]
)

model.save(model_save_path+'/model.h5')
# model.save(model_save_path+'/model.h5py')

Epoch 1/1000
56/85 [==================>...........] - ETA: 0s - loss: 1.7603 - accuracy: 0.2390
Epoch 1: val_loss improved from inf to 1.43123, saving model to ../assets/models/model.h5
85/85 [==============================] - 1s 6ms/step - loss: 1.7114 - accuracy: 0.2614 - val_loss: 1.4312 - val_accuracy: 0.4114
Epoch 2/1000
82/85 [===========================>..] - ETA: 0s - loss: 1.3984 - accuracy: 0.4217
Epoch 2: val_loss improved from 1.43123 to 1.04179, saving model to ../assets/models/model.h5
85/85 [==============================] - 0s 3ms/step - loss: 1.3959 - accuracy: 0.4224 - val_loss: 1.0418 - val_accuracy: 0.7256
Epoch 3/1000
64/85 [=====================>........] - ETA: 0s - loss: 1.1742 - accuracy: 0.5360
Epoch 3: val_loss improved from 1.04179 to 0.78424, saving model to ../assets/models/model.h5
85/85 [==============================] - 0s 3ms/step - loss: 1.1497 - accuracy: 0.5471 - val_loss: 0.7842 - val_accuracy: 0.9200
Epoch 4/1000
76/85 [=========================>.

In [15]:
# Model evaluation

val_loss, val_acc = model.evaluate(x_test, y_test, batch_size)

29/29 [==============================] - 0s 2ms/step - loss: 0.0266 - accuracy: 0.9947


In [5]:
# Loading saved model

model = tf.keras.models.load_model(model_save_path+'/model.h5')

# model.summary()

from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
# Save as a model dedicated to inference
# model.save(model_save_path+'/model_include_optimize.h5', include_optimizer=False)

# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

### Confusion Matrix

In [ ]:
def print_confusion_matrix(y_true, y_pred, report = True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred)
    df_cmx = pd.DataFrame(cmx_data, index = labels, columns = labels)
    fig,ax = plt.subplots(figsize=(7,6))
    sns.heatmap(df_cmx, annot=True, fmt='g', square=False)
    ax.set_ylim(len(set(y_true)),0)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    plt.show()

    if report:
        print('Classification Report')
        print(classification_report(y_validation, y_pred))

In [ ]:
Y_pred = model.predict(x_validation)
y_pred = np.argmax(Y_pred, axis =1)

print_confusion_matrix(y_validation, y_pred)

### Model Plots 

In [ ]:
from matplotlib.pyplot import title


pyo.init_notebook_mode()

def Plot(history , name , model):
    epochs = range(1,len(history.history['loss']) + 1)
    epochs = list(epochs)
    fig = make_subplots(rows=2, 
                        cols=2,
                        subplot_titles=("Train Loss", 
                                        "Train Accuracy",
                                        "Validation Loss", 
                                        "Validation Accuracy",)
                        )
    fig.add_trace(go.Scatter(x=epochs, y=history.history['loss']), row=1, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['accuracy']), row=1, col=2)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['val_loss']), row=2, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=history.history['val_accuracy']), row=2, col=2)
    fig.update_layout(showlegend=False,height=1000, width=1200, title_text=name)
    fig.update_xaxes(title_text='epochs')
    fig.update_yaxes(title_text='value')
    pyo.iplot(fig, filename = 'Act_train_rec')

Plot(history, 'Model valutation',model)